In [61]:
#get IBD rank
#http://myibd.investors.com/search/searchresults.aspx?Ntt=thor
#<a href="http://myibd.investors.com/search/searchresults.aspx?Ntt=Thor Industries Inc&amp;source=manager&amp;in_dym=true" id="ctl00_ctl00_secondaryContent_leftContent_SearchResultsMgrCtrl_didYouMeanCompanyRepeater_ctl00_didYouMeanLink">Thor Industries Inc</a>
#1st: <span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">1</span>
#not 1st: <span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">7</span>
#<a class="stockRoll" href="http://research.investors.com/stock-quotes/nasdaq-alarmcom-holdings-inc-alrm.htm" rel="/stocksymbol.axd?symbol=ALRM">ALRM</a>
#IBD group:
#<div class="spansubHead">Bldg-Mobile/Mfg &amp; Rv Group</div>
#<a class="stockRoll" href="http://research.investors.com/stock-quotes/nasdaq-alarmcom-holdings-inc-alrm.htm" rel="/stocksymbol.axd?symbol=ALRM">ALRM</a>
#<span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">1</span>

#https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/research?symbol=LRCX&appCode=
#https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/financials?stockspage=financials&symbol=LRCX
#https://research2.fidelity.com/fidelity/research/reports/pdf/getReport.asp?feedId=596&docTag=60123&version=20171020
#<a href="http://research2.fidelity.com/fidelity/research/reports/pdf/getReport.asp?feedId=596&amp;docTag=60123&amp;version=20171020" target="_blank">
								<span class="bold">S&amp;P Capital IQ<sup>®</sup> Company Report</span> (PDF)
							</a>

In [95]:
import requests
from bs4 import BeautifulSoup as BSoup
import pandas as pd

In [96]:
#download the page
#https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=financials&symbols=LRCX&period=quarterly  |  ?stockspage=cashflow
#http://research2.fidelity.com/fidelity/research/reports/pdf/getReport.asp?feedId=596&docTag=60123&version=20171013
r = requests.get("https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols={}&period=quarterly".format ("LRCX"))
#now the html code is in the requests object:
print (r.url)
#BeautifulSoup can parse html code and convert into something usable
soup =  BSoup (r.content, "lxml")
#e.g. print html code in readable format
#print (soup.prettify())

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=LRCX&period=quarterly


In [101]:
#<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2017</span> (06/30 - 03/31) </th>
dt=soup.find ("table", {"class": "datatable-component"})#, {"class":"top-bottom-border col-96PX lft-borde"})
#the summary attribute contains a short description of the table, check that this is the Net Income table
#print ("net income" in dt.get("summary").lower ())
#find all rows in this table; the first row contains the header with the dates
allrows = dt.findChildren (name = "tr")
firstrow=(allrows[0])
print (firstrow)
#get the dates on the table:
cols = []
for f in firstrow.find_all ("th"):
    cols.append("{}/{}".format(f.text[6:11], f.text[:4]))
#print ((dt.findChildren(name="td")))
alldatarows = (allrows[1:])
idxs= ["0",]
for row in allrows[1:]:
    f = row.find ("th")
    if f:
        idxs.append (f.text)
new_table = pd.DataFrame(columns=cols, index = idxs) # I know the size 
print (new_table.columns, new_table.index)
   
row_marker = 0
for row in dt.find_all('tr'):
    #print ("row=",row)
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        #print ("col=",column)
        print (row_marker, column_marker, column.get_text())
        if not "Log in" in column.get_text():
            new_table.iat[row_marker,column_marker] = column.get_text()
        else:
            new_table.iat[row_marker, column_marker] = 0.0
        column_marker += 1
    row_marker += 1
        
print (new_table)
#print (pd.read_html(dt))
#print (help(dt))
#table = pd.read_html("https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols={}&period=quarterly".format ("LRCX"))
#print (table)
#for i in dt:
    #if "Net Income" in i.summary:
  #  print (pd.read_html(i))

<tr>
<td class="top-bottom-border col-300PX"></td>
<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2017</span> (09/30 - 06/30) </th>
<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2017</span> (06/30 - 03/31) </th>
<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2017</span> (03/31 - 12/31) </th>
<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2016</span> (12/31 - 09/30) </th>
<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2016</span> (09/30 - 06/30) </th>
</tr>
Index(['09/30/2017', '06/30/2017', '03/31/2017', '12/31/2016', '09/30/2016'], dtype='object') Index(['0', 'Sales/Turnover (Net)', 'Cost of Goods Sold',
       'Gross Profit (Loss)', 'Selling, General and Administrative Expenses',
       'Operating Income Before Depreciation',
       'Depreciation and Amortization – Total\t\t',
       'Operating Income After Deprecia

In [16]:
#now can find all <a>-tags
#soup.find_all ("a")
#get all the hyperlinks
#for link in soup.find_all("a"):
#    print (link.get ("href"))
    #the link text
    #link.text
    #<span id="ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany" onclick="javascript:window.open('http://research.investors.com/stock-quotes/nasdaq-atlassian-corp-plc-cl-a-team.htm')">Atlassian Corp Plc Cl A</span>

In [17]:
#can find the link to the stock's page
stock_page = soup.find_all ("a", {"id": "ctl00_ctl00_secondaryContent_leftContent_SearchResultsMgrCtrl_didYouMeanCompanyRepeater_ctl00_didYouMeanLink"})
if not stock_page:
    print (stock_page)
for link in stock_page:
    print (link.get ("href"), link.text)

[]


In [18]:
#help(stock_page[0])

In [19]:
#print (stock_page[0], "\n")
#print (stock_page[0].get_text(), "\n")
#print (stock_page[0].get('href'))

In [20]:
#follow down the link
#r2=requests.get(stock_page[0].get('href'))

In [22]:
soup2 = BSoup (r.content, "lxml")
#<span id="ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany" onclick="javascript:window.open('http://research.investors.com/stock-quotes/nyse-thor-industries-inc-tho.htm')">Thor Industries Inc</span>
stock_page2 = soup2.find ("span", {"id": "ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany"})
print (stock_page2.get ('onclick'))
tokens = stock_page2.get ('onclick').split ("'")
for t in tokens:
    if "http://" in t:
        stock_link = t
        break
r3 = requests.get (stock_link)
soup3 = BSoup (r3.content, "lxml")
#can find all <span>'s with a certain id:

#spans_id1 = soup3.find_all ("span")
#there is only one occurrence of this (should verify?)
spans_id1 = soup3.find ("span", {"id":"ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank"})
print (spans_id1.text)

#current rank:
#<div id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_currentRank" class="divSymbolRank"><span class="rank-left">#</span><span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_spanSymbolRank"><span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">4</span></span></div>

#first rank:
group_no1 = soup3.find ("a", {"class": "stockRoll"})
#print (len(group_no1))
print (group_no1)

market_group = soup3.find ("div", {"class": "spansubHead"})
print (market_group.text)


javascript:window.open('http://research.investors.com/stock-quotes/nasdaq-advanced-energy-inds-aeis.htm')
1
None
Elec-Semiconductor Equip Group


In [106]:
#    import PyPDF2
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

ModuleNotFoundError: No module named 'pdfminer'